# Prerequisite Installation

In [ ]:
!git clone https://github.com/richzhang/colorization.git

In [ ]:
#!pip uninstall typing-extensions  # Uninstall the conflicting version
#!pip install typing-extensions==3.10.0.0  # Install the version required by Gradio
!pip install --upgrade gradio  # Install or upgrade Gradio

In [ ]:
# !pip install -U gradio

In [ ]:
cd colorization

/content/colorization


In [ ]:
#!pip install -r requirements.txt

In [ ]:
#pip install scikit-image

In [ ]:
pip install torch scikit-image numpy matplotlib argparse Pillow

# Init Code

In [ ]:
import os
from os.path import isfile, join

In [ ]:
import colorizers
colorizer_eccv16 = colorizers.eccv16().eval()
colorizer_siggraph17 = colorizers.siggraph17().eval()

Downloading: "https://colorizers.s3.us-east-2.amazonaws.com/colorization_release_v2-9b330a0b.pth" to /root/.cache/torch/hub/checkpoints/colorization_release_v2-9b330a0b.pth
100%|██████████| 123M/123M [00:03<00:00, 39.4MB/s]
Downloading: "https://colorizers.s3.us-east-2.amazonaws.com/siggraph17-df00044c.pth" to /root/.cache/torch/hub/checkpoints/siggraph17-df00044c.pth
100%|██████████| 130M/130M [00:02<00:00, 45.9MB/s]


# Video processing


In [ ]:
# Importing the necessary packages
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os
from os.path import isfile, join
import subprocess

In [ ]:
def convert_video(vpath):
  vs = cv2.VideoCapture(vpath)
  count = 0
  success = True

  while success:
      success, frame = vs.read()

      if frame is None:
          break

      cv2.imwrite("/content/frames/frame%d.jpg" % count, frame)

      # Call the colorization script for each frame
      subprocess.run([
          'python', '/content/demo_release.py',
          '-i', f'/content/frames/frame{count}.jpg',
          '-o', f'/content/output_frames/output{count}.png'
      ])

      #colorized_frame = cv2.imread(f'content/output_frames/output{count}.png')


      count += 1

      key = cv2.waitKey(1) & 0xFF
      if key == ord("q"):
          break

  vs.release()
  cv2.destroyAllWindows()


In [44]:
def convert_frames_to_video(pathIn, pathOut, fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]

    # for sorting the file names properly
    files.sort(key=lambda x: int(x[5:-4]) if x[5:-4].isdigit() else 0)

    for i in range(len(files)):
        filename = os.path.join(pathIn, files[i])
        print(f"Reading {filename}")
        img = cv2.imread(filename)

        if img is not None:
            height, width, layers = img.shape
            size = (width, height)
            frame_array.append(img)
        else:
            print(f"Unable to read image: {filename}")

    out = cv2.VideoWriter(pathOut, cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(frame_array)):
        out.write(frame_array[i])

    out.release()
    return pathOut


In [59]:
import os
from os.path import isfile, join
import cv2

def convert_frames_to_video(pathIn, pathOut, fps):
    frame_array = []

    # Get a list of files in the input folder
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    #print(files)
    # Sort the files based on the numeric part (Y) in the middle of the filename
    files.sort(key=lambda x: int(x.split('output')[1].split('.png_')[0]))
    #print(files)
    for i in range(len(files)):
        filename = os.path.join(pathIn, files[i])
        print(f"Reading {filename}")
        img = cv2.imread(filename)

        if img is not None:
            height, width, layers = img.shape
            size = (width, height)
            frame_array.append(img)
        else:
            print(f"Unable to read image: {filename}")

    out = cv2.VideoWriter(pathOut, cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(frame_array)):
        out.write(frame_array[i])

    out.release()
    return pathOut

# # Example usage
# input_folder = '/content/output_frames/eccv/'
# output_video = '/content/output_frames/eccv_video.avi'
# convert_frames_to_video(input_folder, output_video, 24)


In [62]:
def move_files(input_folder,eccv_path,siggraph_path):
  import re
  import shutil
  import os
  eccv_folder = '/content/output_frames/eccv/'
  siggraph_folder = '/content/output_frames/siggraph/'

  # Create output folders if they don't exist
  os.makedirs(eccv_folder, exist_ok=True)
  os.makedirs(siggraph_folder, exist_ok=True)

  # Define the pattern for eccv and siggraph files
  eccv_pattern = r"_eccv\d+\.png"
  siggraph_pattern = r"_siggraph\d+\.png"

  # Loop through files in the input folder
  for filename in os.listdir(input_folder):
      file_path = os.path.join(input_folder, filename)

      # Check if the file matches the eccv pattern
      if re.search(eccv_pattern, filename):
          shutil.move(file_path, os.path.join(eccv_folder, filename))
          print(f"Moved {filename} to eccv folder")

      # Check if the file matches the siggraph pattern
      elif re.search(siggraph_pattern, filename):
          shutil.move(file_path, os.path.join(siggraph_folder, filename))
          print(f"Moved {filename} to siggraph folder")

      # If the file doesn't match either pattern, you can choose to skip or handle it differently
      else:
          print(f"File {filename} doesn't match either pattern")

  print("Categorization complete.")




In [68]:
def colorize_video(vpath):
  #print(option)
  # print(vpath)
  convert_video(vpath)
  move_files('/content/output_frames','/content/output_frames/eccv/','/content/output_frames/siggraph/')
  pathIn_eccv= '/content/output_frames/eccv'
  pathIn_sigg= '/content/output_frames/siggraph'
  pathOut_eccv = '/content/video_eccv.avi'
  pathOut_sigg = '/content/video_sigg.avi'
  fps = 30.0
  video_eccv = convert_frames_to_video(pathIn_eccv, pathOut_eccv, fps)
  video_sigg = convert_frames_to_video(pathIn_sigg, pathOut_sigg, fps)
  return video_eccv,video_sigg

In [ ]:
# v1, v2 = colorize_video('/content/output (2).mp4')

# Front end (streamlit)

In [ ]:
import gradio as gr

def generate_output(user_fun,ipath,vpath):
    print(user_fun)
    # Implement your image processing logic here based on user_fun
    # return should be in format - ["image","video","video","video"],


    if user_fun == 'Image Colourization':
      if ipath != None and vpath == None:
        image = colorize_image(ipath)
        return image
      elif ipath == None and vpath != None:
        video1,video2 = colorize_video(vpath)
        return None,video1,video2
    elif user_fun == 'Image Restoration':
      pass
    elif user_fun == 'Image Enhancement':
      pass

iface = gr.Interface(
    generate_output,
    [
        gr.Radio(["Image Restoration", "Image Enhancement", "Image Colourization"], label="Choose a function"),
        gr.Image(label="Upload Image"),
        gr.Video(label="Upload Video") # Use 'sources' to specify upload functionality
    ],
    outputs = ["image","video","video"],
    live=True,
)

iface.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://2c758fe72833e7b441.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Image Colourization


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 495, in call_prediction
    output = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1570, in process_api
    data = self.postprocess_data(fn_index, result["prediction"], state)
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1397, in postprocess_data
    self.validate_outputs(fn_index, predictions)  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1371, in validate_outputs
    raise ValueError(
ValueError: An event handler (generate_output) didn't receive enough output values (needed: 3, received: 1).
Wanted outputs:
    [image, video, video]
Received outputs:
    [None]


Image Colourization
